In [60]:
#  Copyright (c) 2020.
#  Alireza Ghasemieh
#  a.ghasemieh65@gmail.com
#  https://github.com/ghasemieh

from Bugzilla_API import API_data_extract
from text_processing import preprocessing
import pandas as pd
import sqlite3

# Extract data from Bugzilla website
data_df = API_data_extract('1h')

In [61]:
# Preprocess the data_df
data_list = []
for tup in data_df.itertuples():
    processed_summary = preprocessing(data_df,tup.id,'summary')
    data_list.append([tup.id,tup.product,tup.component,tup.creation_time,tup.summary,processed_summary,tup.status])
new_data_df = pd.DataFrame(data_list,columns = ["id","product","component","creation_time",
                                                "summary","processed_summary","status"])

In [62]:
# Create db if not exist
def create_table():
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS bug_report_table (summary TEXT,	component TEXT,	"
               "status TEXT, id INTEGER, creation_time TEXT, product TEXT, processed_summary TEXT )")
    connection.commit()
    connection.close()

def insert(id,product,component,creation_time,summary,processed_summary,status):
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("INSERT INTO bug_report_table VALUES(?,?,?,?,?,?,?)",
                   (id,product,component,creation_time,summary,processed_summary,status))
    connection.commit()
    connection.close()

def view():
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM bug_report_table")
    rows = cursor.fetchall()
    connection.close()
    return rows

def delete(id):
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("DELETE FROM bug_report_table WHERE id=?",(id,))
    connection.commit()
    connection.close()

def update(id,processed_summary):
    connection = sqlite3.connect("bug_database.db")
    cursor = connection.cursor()
    cursor.execute("UPDATE bug_report_table SET processed_summary=? WHERE id=?",(processed_summary,id))
    connection.commit()
    connection.close()


In [63]:
# Save into a SQL database
create_table()
for tup in new_data_df.itertuples():
    insert(tup.id,tup.product,tup.component,tup.creation_time,tup.summary,tup.processed_summary,tup.status)

In [70]:
view()

[('1610194',
  'Firefox for Android',
  'Testing',
  '2020-01-19T19:04:59Z',
  'geckodriver cant start session on Firefox preview',
  'gecko driver start session firefox preview',
  'RESOLVED'),
 ('1610195',
  'Firefox for iOS',
  'General',
  '2020-01-19T19:11:58Z',
  'iOS auto correct/predict dictionary is updated even in Private Browsing mode',
  'ios auto correct predict dictionary update',
  'UNCONFIRMED'),
 ('1610196',
  'Core',
  'DOM: Core & HTML',
  '2020-01-19T19:36:57Z',
  '[wpt-sync] Sync PR 21252 - Enforce character restrictions for registerProtocolHandler scheme',
  'wp sync sync pr enforce character',
  'NEW'),
 ('1610197',
  'Firefox',
  'Untriaged',
  '2020-01-19T19:44:12Z',
  'No debugger variable preview when script contains BigInt literal',
  'debugger variable preview script cont',
  'UNCONFIRMED')]

In [69]:
delete(1610197)